In [ ]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
data_folder = os.getcwd()+'\\IO'

# *Condition trees on gpy\_symbols*

*The current tutorial outlines a method to slice symbols by setting up condition trees. This slicing using condition trees can be used to either return the sliced variable as a pandas series/index, but can similarly be written to gams code. Thus, condition trees are an efficient way to work on the database from Python, but can similarly be transferred directly to the gams code.*

### 0: Initialize and ready the database/symbols

*Set up database to work on (from excel):*

In [ ]:
dsheets = {'Production_v': data_folder+'\\IO_v.xlsx', 'Production_p': data_folder+'\\IO_p.xlsx'}
db = ReadData.read_data.main(dsheets,name='GE_data',components=['domstic','trade','HH','tax','invest'])

*inspect the variables, sets, subsets etc. in the database:*

In [ ]:
display('Variables:',db.variables_flat,'Sets:',db.sets_flat)

*Example the variable qD (quantity, demand):*

In [ ]:
db.get('qD')

### 1: Condition trees

*The condition tree methods are generally build using nested dictionaries:* 
* *The keyword in the dictionary specifies the relevant operation used in the current nest ('and','or','not').*
* *The value specifies the boolean operation to perform. If more than one set of boolean conditions are given they should be defined in a list. (these can be specified in a number of ways)*
* *The boolean operations are specified based on the domains of the relevant symbols. As the examples show, this entails that we do not pass a vector of true/false; rather, we define the condition as a gpy\_symbol that is defined over a certain domain. The domains of the variable in question are then sliced according to the intersection of these domains.*
* *The condition trees can be nested by letting a value in one dictionary, be defined as a new dictionary.*


#### *1.1: Type of arguments*

*The condition tree can be build with three keywords ('and','or','not'). The values, specified in a list if more than one is applied, are generally defined as gpy\_symbols. The interpretation from a gpy\_symbol to a boolean condition is, however, quite flexible. Some examples:*

*1: Simple slicing using an index: The set 'sfor\_ndom' is a set defined over the same index (s,n) as the variable 'qD'.*

In [ ]:
c = db['sfor_ndom']
display('The set:',c.vals, 'The sliced variable:',db['qD'].rctree_pd(c))

*2: Slicing using a partially overlapping index: The set 'n\_prod' is only defined over the set 'n'. The condition tree method looks for the overlap in domains and slice on this condition.*

In [ ]:
c = db['n_prod']
display('The set:',c.vals, 'The sliced variable:',db['qD'].rctree_pd(c))

*3: Slicing using another variable: Similarly, a variable can be used to slice - this is done using the domains of the relevant variable.*

In [ ]:
c = db['qS']
display('The slicing variable:',c.vals, 'The sliced variable:',db['qD'].rctree_pd(c))

*4: The same conditions applied above can be written to a gams code 'condition' using the rctree\_gams method instead (should use indices instead of variables in the slicing though)*

In [ ]:
c = db['n_prod']
db['qD'].rctree_gams(c)

#### *1.2: Negate conditions*

*The simplest condition tree is to negate a condition. This is simply done by adding a dictionary with the keyword 'not':*

In [ ]:
c = {'not': db['n_prod']}
db['qD'].rctree_pd(c)

*We can also just add the value to a list:*

In [ ]:
c = {'not': [db['n_prod']]}
db['qD'].rctree_pd(c)

*It is however not unambiguous how to interpret a list of more than one condition with the keyword not (should it be the union or intersection of 'True' to subset on?). With the keyword 'not', only the first argument of the list is used. Thus, this condition is the same as the one above:*

In [ ]:
c = {'not': [db['n_prod'],db['n_for']]}
db['qD'].rctree_pd(c)

#### *1.3: List of and/or conditions*

*Assume instead that we want to subset on either one subset or the other. This is done with the keyword argument 'or':*

In [ ]:
c = {'or': [db['n_prod'],db['n_for']]}
db['qD'].rctree_pd(c)

*Using the keyword 'and' this returns an empty pandas series, as the two subsets ('n\_prod', 'n\_for') are not overlapping:*

In [ ]:
c = {'and': [db['n_prod'],db['n_for']]}
db['qD'].rctree_pd(c)

*As was the case earlier, this type of conditions can be written as gams code as well:*

In [ ]:
db['qD'].rctree_gams(c)

#### *1.4: Nested conditions*

*Maybe we want the goods that are in one subset, but not the other. In this case, we make an 'and' condition, and nest the 'not' condition for the one subset. The condition tree traverses the nested dictionary to the most inner condition that are no longer nested, evaluate this to return a boolean condition, and loops out of the tree in this fashion. The following lookgs for goods that are in one set 'd\_tauD', but not in another 'd\_tauS':*

In [ ]:
c = {'and': [db['d_tauD'], {'not': db['d_tauS']}]}
db['qD'].rctree_pd(c)

*Just as before, we can write the corresponding gams condition using the rctree\_gams method:*

In [ ]:
db['qD'].rctree_gams(c)

The union of two sets, except elements from a third one:

In [ ]:
c = {"and":[db['n_prod'], db["d_tauD"], {"not":db["d_tauS"]}]}
db["qD"].rctree_pd(c)